In [30]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census API Keys
from config import (api_key)
c = Census(api_key, year=2013)

### The poverty numbers in the dataframe is "Number of persons whose income in the past 12 months is below the poverty level"

In [31]:
#"B25077_001E": "Median Home Value"

census_data = c.acs5.get(("NAME",
                          "B01003_001E",
                          "B17001_002E", 
                          "B17001A_002E", "B17001B_002E", 
                          "B17001D_002E", "B17001I_002E"), 
                         {'for': 'county:*','in':'state:19'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B17001A_002E": "Poverty White",
                                      "B17001B_002E": "Poverty Black", 
                                      "B17001D_002E": "Poverty Asian", 
                                      "B17001I_002E": "Poverty Hispanic",
                                      "NAME": "Name"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Total Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)
#census_pd["White Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)
#census_pd["Black Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)
#census_pd["Asian Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)
#census_pd["Hispanic Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Name", "Population",
                       "Poverty Count", "Poverty White", "Poverty Black", 
                       "Poverty Asian", "Poverty Hispanic", "Total Poverty Rate (%)"]]



split = census_pd["Name"].str.split(",", n = 1, expand = True)
census_pd["County"]= split[0]
census_pd["State"]=split[1]
census_pd = census_pd[["State", "County", "Population", "Poverty Count", 
                       "Poverty White", "Poverty Black", "Poverty Asian", 
                       "Poverty Hispanic", "Total Poverty Rate (%)"]]

census_pd = census_pd.set_index("State")
census_pd.to_csv("2012_Poverty_Data")
census_pd.head()

,County,Population,Poverty Count,Poverty White,Poverty Black,Poverty Asian,Poverty Hispanic,Total Poverty Rate (%)
State,,,,,,,,
Iowa,Adair County,7588.0,704.0,660.0,3.0,12.0,0.0,9.277807
Iowa,Adams County,3973.0,442.0,407.0,5.0,0.0,16.0,11.125094
Iowa,Allamakee County,14242.0,1740.0,1507.0,34.0,11.0,284.0,12.217385
Iowa,Appanoose County,12808.0,1924.0,1854.0,3.0,0.0,68.0,15.021861
Iowa,Audubon County,6002.0,554.0,526.0,0.0,0.0,13.0,9.230257
